In [2]:
!pwd

/home/yongsu/aspect


In [45]:
import pandas as pd
import numpy as np
from sklearn.decomposition import LatentDirichletAllocation
#from nltk.tokenize import word_tokenize, sent_tokenize
# from gensim import corpora
# from gensim.models.ldamodel import LdaModel
import torch
import json, nltk, pickle, ast
#nltk.download('punkt')

### Amazon review dataset specification

> data[0] <br>
    {<br>
    'asin': '0528881469',<br>
     'helpful': [0, 0],<br>
     'overall': 5.0,<br>
     'reviewText': 'We got this GPS for my husband who is an (OTR) over the road trucker.  Very Impressed with the shipping time, it arrived a few days earlier than expected...  within a week of use however it started freezing up... could of just been a glitch in that unit.  Worked great when it worked!  Will work great for the normal person as well but does have the "trucker" option. (the big truck routes - tells you when a scale is coming up ect...)  Love the bigger screen, the ease of use, the ease of putting addresses into memory.  Nothing really bad to say about the unit with the exception of it freezing which is probably one in a million and that\'s just my luck.  I contacted the seller and within minutes of my email I received a email back with instructions for an exchange! VERY impressed all the way around!',<br>
     'reviewTime': '06 2, 2013',<br>
     'reviewerID': 'AO94DHGC771SJ',<br>
     'reviewerName': 'amazdnu',<br>
     'summary': 'Gotta have GPS!',<br>
     'unixReviewTime': 1370131200<br>
    }

### Pre-processing data from extracted aspects

In [46]:
dataset_name = 'electronics'
file_name = '{}_with_aspects2.pkl'.format(dataset_name)
data = pickle.load(open('./raw_data/{}'.format(file_name), 'rb'))
save_dir = './data/{}/'.format(dataset_name)

# Go over each review
'''
[{'rating': 4,
  'item': 'B000HJ75MK',
  'user': 'A37021SSH9LGY6',
  'sentence': [('mouse',
    'light',
    'lightweight mouse to take on a trip overseas',
    1)], # [(ASPECT, SENTIMENT-WORD, RELATED-SENTENCE, SENTIMENT-SCORE), ...]
  'text': 'Mighty little mouse\nI ... '
  }, ... 
]
'''

df_data = pd.DataFrame(data)
df_data_flatten = df_data.explode('sentence')

# Extract aspects from a['sentence'][0]
# aspects = set([ a['sentence'][0] if not pd.isna(a['sentence']) else continue for i, a in df_data_flatten.iterrows() ])
# aspects = list(set([ a['sentence'][0] for i, a in df_data_flatten.iterrows() if not pd.isna(a['sentence']) ]))
aspects = []
for i, a in df_data_flatten.iterrows():
    if not pd.isna(a['sentence']):
        aspects.appenda['sentence'][0]

aspects = ['<sos>', '<eos>', '<pad>', '<unk>'] + aspects
aspect2idx = { a:idx for idx, a in enumerate(aspects) }
idx2aspect = { idx:a for idx, a in enumerate(aspects) }

u_ids = list(df_data['user'].unique())
user2idx = { u:idx for idx, u in enumerate(u_ids)}
idx2user = { idx:u for idx, u in enumerate(u_ids)}
             
i_ids = list(df_data['item'].unique())
item2idx = { i:idx for idx, i in enumerate(i_ids)}
idx2item = { idx:i for idx, i in enumerate(i_ids)}

print('save_dir: {}'.format(save_dir))
pickle.dump(aspect2idx, open(save_dir + 'aspect2idx.pkl', 'wb'))
pickle.dump(idx2aspect, open(save_dir + 'idx2aspect.pkl', 'wb'))
pickle.dump(user2idx, open(save_dir + 'user2idx.pkl', 'wb'))
pickle.dump(idx2user, open(save_dir + 'idx2user.pkl', 'wb'))
pickle.dump(item2idx, open(save_dir + 'item2idx.pkl', 'wb'))
pickle.dump(idx2item, open(save_dir + 'idx2item.pkl', 'wb'))


AttributeError: 'list' object has no attribute 'appenda'

In [47]:
'''
    a u/i review -- a sequence of words which are assigned to one of topics each
    // e.g., 
    "topic.pkl": topic2idx dictionary, including <sos>, <eos>, <unk>, <pad>, and topic labels. 
    // {TOPIC-IDX: (TOPIC-WORD-IDX1, TOPIC-WORD-IDX2, ...)} e.g., {0: (1,1002,1123, ...), }
    "topic_rev.pkl": idx2topic dictionary, the reverse of topic2idx
    "user.pkl" and "item.pkl": user2idx and item2idx dictionary
'''

SyntaxError: invalid syntax (<ipython-input-47-661e3c02d3e6>, line 10)

### Split train/val/test dataset

In [47]:
from sklearn.model_selection import train_test_split

random_state = 42
file_to_split = 'electronics_with_aspects2'

# Load original review dataset (100k sampled)
# Dataset is not really jsonized one -- json elements are line-separated
# with open('./raw_data/{}.json'.format(file_to_split), 'r') as f:
#     all_data = [ eval(l) for l in f.readlines() ]
    
# Load files for extracted aspects
all_data = pickle.load(open('./raw_data/{}.pkl'.format(file_to_split), 'rb'))

# Remove length-0 or -1 aspect sequences
all_data_filtered = []
for r in all_data:
    if ('sentence' in r.keys()) and (len(r['sentence']) > 1):
        all_data_filtered.append(r)

# Remove length-0 or -1 aspect sequences
# all_data_filtered = [ r for r in all_data if 'sentence' in r.keys() ]
    
train_ratio = 0.8
validation_ratio = 0.1
test_ratio = 0.1

x_train, x_test = train_test_split(all_data_filtered, test_size=1 - train_ratio, random_state=random_state)
x_val, x_test = train_test_split(x_test, test_size=test_ratio/(test_ratio + validation_ratio), random_state=random_state)

# Save splitted train/test/valid jsons back to file
with open('./data/electronics/train.json', 'w') as f:
    for r_dict in x_train:
        f.write(json.dumps(r_dict))
        f.write('\n')
    f.close()
    
with open('./data/electronics/test.json', 'w') as f:
    for r_dict in x_test:
        f.write(json.dumps(r_dict))
        f.write('\n')
    f.close()
    
with open('./data/electronics/validation.json', 'w') as f:
    for r_dict in x_val:
        f.write(json.dumps(r_dict))
        f.write('\n')
    f.close()
    


In [48]:
len(x_train)

94429

In [37]:
random_state = 42
file_to_split = 'electronics_with_aspects'

# Load original review dataset (100k sampled)
# Dataset is not really jsonized one -- json elements are line-separated
# with open('./raw_data/{}.json'.format(file_to_split), 'r') as f:
#     all_data = [ eval(l) for l in f.readlines() ]
    
# Load files for extracted aspects
all_data = json.load(open('./raw_data/{}.json'.format(file_to_split), 'r'))
all_data_filtered = [ r for r in all_data if 'sentence' in r.keys() ]


In [11]:
np.histogram([len(r['sentence']) for r in all_data_filtered], bins=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11])

(array([    0, 32666, 14780,  6553,  3109,  1590,   941,   525,   333,
          231,   213]),
 array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11]))

In [38]:
len(all_data_filtered)

61187

In [12]:
len(all_data)

118312

In [30]:
for r in x_train:
    if r['user'] == 'A2OS3TIVAKUAHG':
        print(r)

{'text': 'doesn\'t work with iphone 4 bumper- but price is right\nThe Philips DC291 has an impressive list of features that had me quite interested.  Having iPhones, iPods, and an iPad 2, the compatibility with Apple products was a major draw.  Overall I was pleased with how these features worked.  There are some flaws (which I will detail below) but for the relatively low price, I\'m willing to forgive these failings.PROsHandsome alarm clock is sleek and modern looking.Sound is very good and very loud for the size.The included remote control works well for basic navigation of your playlist.Time is automatically set when your iPhone/iPad/whatever is plugged into it.CONsThe numbers in the time display on the LCD screen are too small (&lt; 1 inch tall).The viewing angle of the LCD display is really bad.iPhone 4 will not dock with Apple Bumper installed.iPhone 4 gives sporadic error messages when docked.iPhone 4---------------The first device that I tried was my iPhone 4.  Mine has the Ap

In [27]:
user2idx['A2K7ZSSBUHAQ5O']

2214

In [28]:
idx2user[344]

'A2OS3TIVAKUAHG'

In [31]:
idx2item[34869]

'B006SOLITY'

In [33]:
idx2aspect[816]

'booster'